# Lab Rotation

In [1]:
import pandas as pd
import re as re
import numpy as np
import requests
from urllib.parse import urlparse
from time import sleep
import datasets

c:\Users\slim_\miniconda3\envs\labrotation\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# TODO 
github_token = "ADD_YOUR_GITHUB_TOKEN"

## Data Collection
@Misc{acl-ocl,
    author =       {Shaurya Rohatgi, Yanxia Qin, Benjamin Aw, Niranjana Unnithan, Min-Yen Kan},
    title =        {The ACL OCL Corpus: advancing Open science in Computational Linguistics},
    howpublished = {arXiv},
    year =         {2022},
    url =          {https://huggingface.co/datasets/ACL-OCL/ACL-OCL-Corpus}
}

In [3]:
df = pd.read_parquet('data/acl-publication-info.74k.v2.parquet')
#df2 = datasets.load_dataset("WINGNUS/ACL-OCL")

df.head()

,acl_id,abstract,full_text,corpus_paper_id,pdf_hash,numcitedby,url,publisher,address,year,...,doi,number,volume,journal,editor,isbn,ENTRYTYPE,ID,language,note
0,O02-2002,There is a need to measure word similarity whe...,There is a need to measure word similarity whe...,18022704,0b09178ac8d17a92f16140365363d8df88c757d0,14,https://aclanthology.org/O02-2002,None,None,2002,...,None,None,None,None,None,None,inproceedings,chen-you-2002-study,None,None
1,L02-1310,None,None,8220988,8d5e31610bc82c2abc86bc20ceba684c97e66024,93,http://www.lrec-conf.org/proceedings/lrec2002/...,European Language Resources Association (ELRA),"Las Palmas, Canary Islands - Spain",2002,...,None,None,None,None,None,None,inproceedings,mihalcea-2002-bootstrapping,None,None
2,R13-1042,Thread disentanglement is the task of separati...,Thread disentanglement is the task of separati...,16703040,3eb736b17a5acb583b9a9bd99837427753632cdb,10,https://aclanthology.org/R13-1042,"INCOMA Ltd. Shoumen, BULGARIA","Hissar, Bulgaria",2013,...,None,None,None,None,None,None,inproceedings,jamison-gurevych-2013-headerless,None,None
3,W05-0819,"In this paper, we describe a word alignment al...","In this paper, we describe a word alignment al...",1215281,b20450f67116e59d1348fc472cfc09f96e348f55,15,https://aclanthology.org/W05-0819,Association for Computational Linguistics,"Ann Arbor, Michigan",2005,...,None,None,None,None,None,None,inproceedings,aswani-gaizauskas-2005-aligning,None,None
4,L02-1309,None,None,18078432,011e943b64a78dadc3440674419821ee080f0de3,12,http://www.lrec-conf.org/proceedings/lrec2002/...,European Language Resources Association (ELRA),"Las Palmas, Canary Islands - Spain",2002,...,None,None,None,None,None,None,inproceedings,suyaga-etal-2002-proposal,None,None


In [4]:
df = df.rename(columns={"acl_id": "paper_ident", # unique paper identifier
                   "url": "paper_url", # Paper online abstract page URL.
                   "author": "paper_author", # Author list.
                   "title": "paper_title", # Paper title
                   "journal": "paper_venue", # Venue abbreviation.
                   "year": "paper_year", # Publication year
                   "month": "paper_month", # Publication month.
                   "booktitle": "paper_booktitle", # BibTeX booktitle field.
                   "address": "paper_address", # BibTeX adress field
                   "publisher": "paper_publisher", # BibTeX publisher field      
                   "pages": "paper_pages", # BibTeX pages.
                   "full_text": "paper_text",
                   })

df = df.drop(columns=["abstract", "corpus_paper_id", "pdf_hash", "doi",
                              "numcitedby", "number", "volume",  
                              "editor", "isbn", "ENTRYTYPE","ID", "language", "note"])

In [5]:
df['error_download'] = df['paper_text'].apply(lambda x: not x.strip() if isinstance(x, str) else True)

df['error_download'].value_counts()

error_download
False    67414
True      5871
Name: count, dtype: int64

# Meteor Scores
## Reproducibility

### METEOR Identification - paper_meteor_prelim

In [6]:
df_meteor_prelim = df.copy()
df_meteor_prelim["paper_meteor_prelim"] = df_meteor_prelim["paper_text"].str.contains(" meteor ", case=False)

df_meteor_prelim['paper_meteor_prelim'].value_counts()

paper_meteor_prelim
False    65842
True      1613
Name: count, dtype: int64

### Paper Review
#### METEOR Parameters - paper_meteor_params


Parameters according to https://www.cs.cmu.edu/~alavie/METEOR/README.html

-l language                     Fully supported: en cz de es fr ar
                                Supported with language-independent parameters:
                                  da fi hu it nl no pt ro ru se 
                                  
-t task                         One of: rank util adq hter li tune
                                  util implies -ch

-p 'alpha beta gamma delta'     Custom parameters (overrides default)

-m 'module1 module2 ...'        Specify modules (overrides default)
                                  Any of: exact stem synonym paraphrase

-w 'weight1 weight2 ...'        Specify module weights (overrides default)

-r refCount                     Number of references (plaintext only)

-x beamSize                     (default 40)

-s wordListDirectory            (if not default for language)

-d synonymDirectory             (if not default for language)

-a paraphraseFile               (if not default for language)

-f filePrefix                   Prefix for output files (default 'meteor')

-q                              Quiet: Segment scores to stderr, final to stdout,
                                  no additional output (plaintext only)

-ch                             Character-based precision and recall

-norm                           Tokenize / normalize punctuation and lowercase
                                  (Recommended unless scoring raw output with
                                   pretokenized references)

-lower                          Lowercase only (not required if -norm specified)

-noPunct                        Do not consider punctuation when scoring
                                  (Not recommended unless special case)

-sgml                           Input is in SGML format

-mira                           Input is in MIRA format
                                  (Use '-' for test and reference files)

-vOut                           Output verbose scores (P / R / frag / score)

-ssOut                          Output sufficient statistics instead of scores

-writeAlignments                Output alignments annotated with Meteor scores
                                  (written to <prefix>-align.out)

In [7]:
def extract_parameters(text):
    pattern = r"((?: -[a-z123](?: [a-z0-9.]{1,4})?){2,})"
    matches = re.findall(pattern, text)
    return matches[0] if matches else None

In [8]:
df_meteor_params = df_meteor_prelim.copy()
df_meteor_params["paper_meteor_params"] = df_meteor_params.apply(lambda row: extract_parameters(row['paper_text']) if row['paper_meteor_prelim'] else None, axis=1)

df_meteor_params['paper_meteor_params'].value_counts()

paper_meteor_params
 -s or -e                          2
 -1 -1                             2
 -o -v                             1
 -3 to -2                          1
 -n 2 -m -u -c 95 -x -r 1000 -f    1
 -t -d 0                           1
 -e -a pojk                        1
Name: count, dtype: int64

#### METEOR Protocol - paper_meteor_protocol
- Language: -l language
    - Use settings for specified language. Lang can be either the language name or two letter code. See the supported language list.
    - Fully supported: en cz de es fr ar
    - Supported with language-independent parameters: fi hu it nl no pt ro ru se tr
    
- Task: -t task
    - Use a different pre-defined set of parameters for scoring (currently limited to English):
        - rank: parameters tuned to human rankings from WMT09 and WMT10
        - adq: parameters tuned to adequacy scores from NIST Open MT 2009
        - hter: parameters tuned to HTER scores from GALE P2 and P3
        - li: language-independent parameters
    - One of: rank util adq hter li tune (util implies -ch)

- Modules: -m 'module1 module2 ...'
    - Set modules manually. Options are: exact stem synonym paraphrase. See supported languages. Module string should be quoted. 

- Normalize: -norm
    - Tokenize and lowercases input lines, normalize punctuation to improve scoring accuracy. This option is highly recommended unless scoring raw system output against pretokenized references. 

- Lowercase: -lower
    - Lowercase input lines (not required if -norm also specified). This is most commonly used scoring cased, tokenized outputs with pretokenized references. 

- Ignore Punctuation: -noPunct
    - If specified, punctuation symbols will be removed before scoring. This is generally not recommended as parameters are tuned with punctuation included. 

- Character-based -ch
    - Calculate character-based precision and recall. Alignment is still word and phrase-level. Fragmentation penalty is still word and phrase-level. 

- Verbose Output: -vOut
    - Output verbose scores (Precision, Recall, Fragmentation, Score) in place of regular scores

In [9]:
regex_meteor_protocol = {
    'rank': r'\b(?:rank|ranking|WMT09|WMT10)\b',
    'adq': r'\b(?:adq|adequacy|NIST Open MT 2009)\b',
    'hter': r'\b(?:hter|HUMAN-targeted translation edit rate|GALE P2|GALE P3)\b',
    'li': r'\b(?:li|language[- ]independent)\b',
    'modules': r'\b(?:-m\s+(?:exact|stem|synonym|paraphrase)|module|exact|stem|synonym|paraphrase)\b',
    'normalize': r'\b(?:-norm|normalize|normalization|tokenize\s+and\s+lowercase|normalize\s+punctuation)\b',
    'lowercase': r'\b(?:-lower|lowercase|lowercasing|casing)\b',
    'ignore_punctuation': r'\b(?:-noPunct|no\s+Punctuation|ignore\s+punctuation|remove\s+punctuation)\b',
    'character_based': r'\b(?:-ch|character[- ]based|calculate\s+character[- ]based\s+precision\s+and\s+recall)\b',
    'verbose_output': r'\b(?:-vOut|verbose\s+output|output\s+verbose\s+scores)\b'
}

In [10]:
def search_terms_near_meteor(text, regex_dict):
    results = []
    for term, pattern in regex_dict.items():
        # Find all occurrences of 'meteor' (case insensitive)
        for match in re.finditer(r'meteor', text, re.IGNORECASE):
            start, end = match.start(), match.end()
            # Define a 500-character window around 'meteor'
            window_start, window_end = max(0, start - 500), min(len(text), end + 500)
            # Search for the term within this window
            if re.search(pattern, text[window_start:window_end], re.IGNORECASE):
                results.append(term)
    return list(set(results))  # Return unique terms

In [11]:
df_meteor_protocol=df_meteor_params.copy()
df_meteor_protocol['paper_meteor_protocol'] = df_meteor_protocol[df_meteor_protocol['paper_meteor_prelim'] == True]['paper_text'].apply(lambda x: search_terms_near_meteor(x, regex_meteor_protocol))

df_meteor_protocol["paper_meteor_protocol"].value_counts().head(20)

paper_meteor_protocol
[]                            947
[modules]                     211
[rank]                         98
[adq]                          62
[li]                           53
[modules, rank]                47
[modules, li]                  23
[adq, modules]                 20
[adq, rank, modules]           13
[adq, rank]                    12
[hter]                         11
[modules, normalize]           11
[normalize]                    10
[rank, li]                     10
[lowercase]                     8
[character_based]               8
[adq, rank, hter, modules]      7
[lowercase, modules]            6
[modules, rank, li]             5
[adq, modules, li]              4
Name: count, dtype: int64

#### METEOR packages - paper_meteor_packages

In [12]:
def search_for_regex_pattern(text, regex_dict):
    results = []
    for term, pattern in regex_dict.items():
        # Search for the pattern in the entire text
        if re.search(pattern, text, re.IGNORECASE):
            results.append(term)
    return list(set(results))  # Return unique terms

In [19]:
regex_meteor_packages = {
    # METEOR versions
    # uncomment this to see an overlap between METEOR versions and packages
    # 'Meteor_1.5': r'Meteor\s+Universal.*?Denkowski.*?Lavie.*?EACL.*?2014',
    # 'Meteor_1.3': r'Meteor\s+1\.3.*?Denkowski.*?Lavie.*?EMNLP.*?2011',
    # 'Meteor_NEXT': r'METEOR(?:-NEXT)?.*?Denkowski.*?Lavie.*?ACL.*?2010',
    # 'Meteor_Phrase_Level': r'Meteor.*?Phrase\s+Level.*?NAACL(?:/HLT)?.*?2010',
    # 'Meteor_MT_2010': r'METEOR.*?Machine\s+Translation(?:\s+Workshop)?.*?2010',
    # 'Meteor_MBleu_MTer': r'METEOR.*?(?:M-BLEU|M-TER).*?ACL.*?2008',
    # 'Meteor_Automatic_MT_2007': r'METEOR.*?Automatic.*?MT\s+Evaluation.*?ACL.*?2007',
    # 'Meteor_Automatic_MT_2005': r'METEOR.*?Automatic.*?MT\s+Evaluation.*?ACL.*?2005',
    # METEOR implementations
    # identified by manual research and
    # code repositories mentioned > 1 times (see code review)
    'Meteor_coco': r'(?:coco.*?meteor|meteor.*?coco)',
    'pymeteor': r'pymeteor|zembrodt',   
    'generationeval_meteor': r'generationeval.*?meteor|meteor.*?generationeval|webnlg.*?meteor|meteor.*?webnlg',
    'evaluatemetric_meteor': r'evaluatemetric.*?meteor|meteor.*?evaluate(?:-)?metric',    
    'fairseq_meteor': r'fairseq.*?meteor|meteor.*?fairseq',
    'nlgeval_meteor': r'nlgeval.*?meteor|meteor.*?nlg(?:-)?eval',
    'nltk_meteor': r'nltk.*?meteor|meteor.*?nltk',
    'meteorscorer': r'meteorscorer',
    #'beer_meteor': r'beer.*?meteor|meteor.*?beer',
    'compare_mt_meteor': r'compare(?:-)?mt.*?meteor|meteor.*?compare(?:-)?mt',
    'pysimt_meteor': r'pysimt.*?meteor|meteor.*?pysimt',
    #'blend_meteor': r'blend.*?meteor|meteor.*?blend',
    'stasis_meteor': r'stasis.*?meteor|meteor.*?stasis',
    'huggingface' : r'hugging\s*face.*?evaluate.*?meteor|meteor.*?hugging\s*face.*?evaluate',
    'SummEval_meteor': r'SummEval.*?meteor|meteor.*?Yale-LILY|Yale-LILY.*?meteor|meteor.*?SummEval',
    'vizseq_meteor': r'vizseq.*?meteor|meteor.*?vizseq|facebookresearch.*?meteor|meteor.*?facebookresearch',
    'github meteor': r'github.com.{0,50}meteor',
    # Template for other versions
    'Meteor_x': r'Meteor\s+[xX]\.?[\d\.]*', 
}

In [20]:
df_meteor_packages = df_meteor_protocol.copy()
# Applying the function to the DataFrame
df_meteor_packages['paper_meteor_packages'] = df_meteor_packages[df_meteor_packages['paper_meteor_prelim'] == True]['paper_text'].apply(lambda x: search_for_regex_pattern(x, regex_meteor_packages))

df_meteor_packages["paper_meteor_packages"].value_counts().head(20)

paper_meteor_packages
[]                                      1250
[Meteor_coco]                            128
[nltk_meteor]                             48
[generationeval_meteor]                   44
[huggingface]                             36
[fairseq_meteor]                          28
[Meteor_coco, fairseq_meteor]              9
[nlgeval_meteor]                           8
[SummEval_meteor]                          6
[huggingface, nltk_meteor]                 5
[huggingface, generationeval_meteor]       4
[Meteor_coco, nlgeval_meteor]              4
[Meteor_coco, nltk_meteor]                 4
[generationeval_meteor, nltk_meteor]       4
[huggingface, fairseq_meteor]              3
[vizseq_meteor]                            2
[Meteor_x]                                 2
[compare_mt_meteor]                        2
[Meteor_coco, generationeval_meteor]       2
[compare_mt_meteor, fairseq_meteor]        2
Name: count, dtype: int64

### Code Review
#### URL of code repository cited in paper - code_meteor_url

In [80]:
df_meteor_url = df_meteor_packages.copy()

# Define regex for matching codebase URLs
regex_codebases = r'https?://(?:www\.)?(?:github\.com|gitlab\.com|bitbucket\.org|sourceforge\.net|google\.code|code\.google)[^\s)]*(?<!\.)'

# Function to extract codebase URLs from text using regex
def extract_codebases(text):
    # Use re.findall to match all instances of the regex in the text
    return re.findall(regex_codebases, text)

# Apply extract_codebases function to 'paper_text', store URLs in a list within each cell
df_meteor_url["code_meteor_url"] = df_meteor_url.apply(
    lambda row: extract_codebases(row['paper_text']) if row['paper_meteor_prelim'] and pd.notnull(row['paper_text']) else [],
    axis=1
)

df_meteor_url["code_meteor_url"].value_counts()

code_meteor_url
[]                                             73021
[https://github.com/]                             35
[http://code.google.com/p/giza-pp/]                4
[https://github.com/google-research/]              3
[https://github.com/UKPLab/]                       2
                                               ...  
[https://github.com/proycon/deepfrog]              1
[https://github.com/QAQ-v/HetGT]                   1
[https://github.com/c-w/gutenberg]                 1
[https://github.com/google-research/pangea]        1
[https://github.com/shiranD/word_]                 1
Name: count, Length: 218, dtype: int64

#### Does the code mention METEOR? - code_meteor_prelim

In [52]:
# Extract GitHub repository names from URLs
def extract_github_repo_names(urls):
    return ["/".join(urlparse(url).path.strip("/").split("/")[:2]) for url in urls if "github.com" in urlparse(url).netloc]

# Apply the function to extract GitHub repository names only if 'paper_meteor_prelim' is True
df_meteor_url['code_meteor_github'] = df_meteor_url.apply(lambda row: extract_github_repo_names(row['code_meteor_url']) if row['paper_meteor_prelim'] else [], axis=1)

In [53]:
df_meteor_code = df_meteor_url.copy()

# Initialize column for search results
df_meteor_code['code_meteor_prelim'] = False

# Collect unique GitHub repositories from papers with 'paper_meteor_prelim' == True
unique_repos = set(repo for repos_list in df_meteor_code[df_meteor_code['paper_meteor_prelim'] == True]['code_meteor_github'] for repo in repos_list)

In [ ]:
df_meteor_packages.to_pickle("meteor_paper_review.pkl")

In [54]:
# Search GitHub repositories for "meteor" keyword
def search_github_repos(unique_repos, token):
    print(f"Starting search for 'meteor' in {len(unique_repos)} unique GitHub repositories...")
    repo_search_results = {}
    headers = {'Authorization': f'token {token}', 'Accept': 'application/vnd.github.v3+json'}
    for idx, repo in enumerate(unique_repos):
        search_url = f"https://api.github.com/search/code?q=meteor+repo:{repo}"
        try:
            response = requests.get(search_url, headers=headers)
            repo_search_results[repo] = response.status_code == 200 and response.json()['total_count'] > 0
        except Exception as e:
            print(f"Error searching {repo}: {e}")
        if (idx + 1) % 10 == 0:
            sleep(60)  # Sleep to respect the GitHub search API rate limit
    return repo_search_results

# Perform the search
repo_search_results = search_github_repos(unique_repos, github_token)

# Update 'code_meteor_prelim' based on the search results
def update_prelim_column(repos):
    if not repos:  # If the list of repositories is empty, do not change the value
        return np.nan
    return any(repo_search_results.get(repo, False) for repo in repos)

# Apply the update function to 'code_meteor_prelim' only for rows where 'paper_meteor_prelim' is True
df_meteor_code['code_meteor_prelim'] = df_meteor_code.apply(lambda row: update_prelim_column(row['code_meteor_github']) if row['paper_meteor_prelim'] else row['code_meteor_prelim'], axis=1)

df_meteor_code['code_meteor_prelim'].value_counts()


Starting search for 'meteor' in 227 unique GitHub repositories...


code_meteor_prelim
False    71867
True        56
Name: count, dtype: int64

In [55]:
df_meteor_code.to_pickle("meteor_code.pkl")

#### Manual Code Review

In [67]:
df_meteor_code_review = df_meteor_code.copy()

In [68]:
# Create a separate DataFrame for GitHub repositories where "meteor" has been found
df_meteor_code_github = pd.DataFrame([repo for repo, found in repo_search_results.items() if found], columns=['github_repository'])

# Export the DataFrame with counts to a CSV file
df_meteor_code_github.to_csv('meteor_github.csv', index=False)

In [69]:
df_meteor_code_manual = pd.read_csv("meteor_code_review.csv", sep=";")

In [70]:
# Explode df_meteor_code to have one repository name per row for merging, ensure 'acl_id' is retained
df_meteor_code_exploded = df_meteor_code.explode('code_meteor_github')

# Merge the manual review data into the exploded dataframe on 'code_meteor_github'
df_merged = pd.merge(
    df_meteor_code_exploded,
    df_meteor_code_manual[['code_meteor_github', 'code_meteor']],
    on='code_meteor_github',
    how='left'
)

# Define aggregation methods for all columns
# 'first' for most columns to avoid duplication, 'max' for 'code_meteor' assuming it's boolean or numerical
aggregation_methods = {col: 'first' for col in df_merged.columns if col not in ['code_meteor_github', 'code_meteor']}
aggregation_methods['code_meteor_github'] = lambda x: list(set(x.dropna()))  # Unique GitHub repository names
aggregation_methods['code_meteor'] = 'max'  # Max to ensure if any true value exists, it prevails

# Group by 'acl_id' and aggregate according to defined methods
df_meteor_code_review = df_merged.groupby('paper_ident', as_index=False).agg(aggregation_methods)

# Optional: Clean up 'code_meteor_github' if necessary, removing any potential empty strings
df_meteor_code_review['code_meteor_github'] = df_meteor_code_review['code_meteor_github'].apply(lambda x: [item for item in x if item])

print(df_meteor_code_review["code_meteor"].value_counts())

code_meteor
True     33
False    13
Name: count, dtype: int64


#### Software error

In [71]:
df_meteor_code_review['software_error'] = False

In [72]:
packages_with_errors = ['nltk_meteor', 'fairseq_meteor', 'huggingface']

# Function to check if any package in the list is contained in the paper_meteor_packages column
def contains_package(packages):
    # Check if 'packages' is not a list (e.g., NaN or None)
    if not isinstance(packages, list):
        return False
    # Check if any of the specified packages with errors are in the 'packages' list
    return any(pkg in packages for pkg in packages_with_errors)

# Apply the function to the paper_meteor_packages column and assign the result to the software_error column
df_meteor_code_review['software_error'] = df_meteor_packages['paper_meteor_packages'].apply(contains_package)

df_meteor_code_review['software_error'].value_counts()

software_error
False    73123
True       162
Name: count, dtype: int64

### Reproducibility
R1: Paper cites METEOR package and parameters.

R2: Paper cites no-config4 METEOR package.

R3: Codebase has complete METEOR evaluation 

In [74]:
df_meteor_reproducible = df_meteor_code_review.copy()

# Initialize the 'reproducible' column as a nullable boolean
df_meteor_reproducible['reproducible'] = False

In [75]:
# R1: Check if both packages and params are not null
condition_r1 = df_meteor_reproducible['paper_meteor_packages'].notna() & df_meteor_reproducible['paper_meteor_params'].notna()
df_meteor_reproducible.loc[condition_r1, 'reproducible'] = True

df_meteor_reproducible['reproducible'].value_counts()

reproducible
False    73276
True         9
Name: count, dtype: int64

In [76]:
# R2: Check for no configuration packages
# Define the no configuration packages list
no_config_packages = ['generationeval_meteor', 'Meteor_coco', 'pymeteor', 'nlgeval_meteor', 'nltk_meteor',
                      'huggingface', 'fairseq_meteor', ]

def check_reproducibility(row):
    # Only modify if reproducible is False or pd.NA
    if row['reproducible'] is False or pd.isna(row['reproducible']):
        # Check if paper_meteor_packages is a list and not empty or NA
        if isinstance(row['paper_meteor_packages'], list) and row['paper_meteor_packages']:
            # Check if any package in the list requires no configuration
            if any(pkg in no_config_packages for pkg in row['paper_meteor_packages']):
                return True
    return row['reproducible']

# Apply the function to update the 'reproducible' column
df_meteor_reproducible['reproducible'] = df_meteor_reproducible.apply(check_reproducibility, axis=1)

df_meteor_reproducible['reproducible'].value_counts()

reproducible
False    72928
True       357
Name: count, dtype: int64

In [77]:
# R3: Check if 'code_meteor' is True
condition_r3 = df_meteor_reproducible['code_meteor'] == True
df_meteor_reproducible.loc[condition_r3, 'reproducible'] = True

df_meteor_reproducible['reproducible'].value_counts()

reproducible
False    72911
True       374
Name: count, dtype: int64

### Export

In [78]:
df_meteor_reproducible["code_meteor"] = df_meteor_reproducible["code_meteor"].apply(lambda x: False if x != True else True).fillna(False)

In [ ]:
df_meteor_reproducible.to_pickle("meteor_review.pkl")

In [79]:
def save_dataset(df: pd.DataFrame, filepath: str = "meteorscores/data/meteor_papers.jsonl.gz") -> None:
    """
    Save the DataFrame to a .jsonl.gz file, excluding the 'paper_text' column.
    
    Parameters:
    - df: The DataFrame to save.
    - filepath: The file path where the DataFrame should be saved.
    """
    try:
        df.drop(columns=['paper_text']).to_json(filepath, orient="records", lines=True, compression="gzip")
        print(f"Dataset successfully saved to {filepath}")
    except Exception as e:
        print(f"Could not save dataset: {e}")

save_dataset(df_meteor_reproducible, "meteorscores/data/meteor_papers.jsonl.gz")

Dataset successfully saved to meteorscores/data/meteor_papers.jsonl.gz
